In [1]:
import os
import math
import numpy as np
import pandas as pd
import random
from IPython.display import display, HTML
from sklearn.preprocessing import OrdinalEncoder
import sys
sys.path.insert(1, '/home/ptr@itd.local/code/fairness_triangle/tools')  # Update this path as needed
from preprocessing import *
from sklearn.utils import resample



In [2]:
random.seed(42) # For reproducibility
sample_df = pd.read_csv("../datasets/2024_public_lar.csv", skiprows=lambda i: i > 0 and random.random() > 0.01)

In [3]:
# Find columns with >35% missing values
missing_ratio = sample_df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > 0.35].index.tolist()
cols_to_drop = cols_to_drop + ['lei', 'activity_year', 'census_tract', 'purchaser_type', 'preapproval', 'state_code','county_code', 'applicant_age_above_62'] 
#print("Columns with >35% missing values:")
#print(cols_to_drop)

# Drop them from the DataFrame
sample_df_clean = sample_df.drop(columns=cols_to_drop)
print(f"Original shape: {sample_df.shape}")
print(f"New shape after drop: {sample_df_clean.shape}")

must_have_val = [
    'co_applicant_ethnicity_1',
    'co_applicant_race_1',
    'loan_amount',
    'property_value',
    'income',
    'debt_to_income_ratio',
    'applicant_credit_score_type',
    'action_taken',
    'aus_1',
    'applicant_race_1',
    'applicant_ethnicity_1',
    'applicant_sex'
]


sample_df_clean = sample_df_clean.dropna(subset=must_have_val)

Original shape: (121729, 99)
New shape after drop: (121729, 58)


In [4]:
low_var_cols = [col for col in sample_df_clean.columns if sample_df_clean[col].nunique() <= 1]
sample_df_clean = sample_df_clean.drop(columns=low_var_cols)

In [5]:
sample_df_clean

,derived_msa_md,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,loan_type,loan_purpose,...,initially_payable_to_institution,aus_1,denial_reason_1,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
2,12580,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Male,1,1,31,...,1,6,10,5988,91.53,122200,96.0,2228,2321,54
3,35084,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Female,3,1,4,...,1,6,1,3661,83.15,133300,78.0,712,1053,68
4,99999,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Joint,1,1,2,...,1,6,10,3598,7.45,95000,130.0,1095,1508,57
7,15764,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Female,3,1,2,...,1,6,1,5925,60.19,146600,60.0,1213,1707,0
9,45060,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Female,1,1,2,...,1,6,10,3642,25.40,94800,76.0,909,1283,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121724,16540,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Female,1,1,4,...,1,6,10,4251,11.08,92700,96.0,1066,1660,54
121725,45060,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Joint,3,1,2,...,1,6,3,3282,15.81,94800,210.0,1120,1288,48
121726,47764,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,Black or African American,Joint,3,1,2,...,1,6,9,4883,52.08,128300,146.0,1439,1570,52
121727,48864,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Joint,1,1,2,...,1,6,10,6561,33.53,111300,123.0,2015,2208,19


In [6]:
#Make Y bar binary (m/w)
sample_df_clean['applicant_sex'].unique()
sample_df_clean = sample_df_clean[sample_df_clean['applicant_sex'].isin([1, 2])]
sample_df_clean['applicant_sex'] = sample_df_clean['applicant_sex'].replace({1: 0, 2: 1})

/tmp/ipykernel_479012/4108010613.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df_clean['applicant_sex'] = sample_df_clean['applicant_sex'].replace({1: 0, 2: 1})


In [7]:
#Make Y binary (loan originated/not originated)
sample_df_clean['action_taken'] = sample_df_clean['action_taken'].apply(lambda x: 1 if x == 1 else 0)
print((sample_df_clean['action_taken'] == 1).mean() * 100)

72.06492089470812


/tmp/ipykernel_479012/1590058750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df_clean['action_taken'] = sample_df_clean['action_taken'].apply(lambda x: 1 if x == 1 else 0)


In [8]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)


72.06492089470812


In [9]:
# Convert loan_term to int and remove NaN
sample_df_clean['loan_term'] = pd.to_numeric(sample_df_clean['loan_term'], errors='coerce')
sample_df_clean = sample_df_clean.dropna(subset=['loan_term'])
sample_df_clean['loan_term'] = sample_df_clean['loan_term'].astype(int)

/tmp/ipykernel_479012/999455300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df_clean['loan_term'] = pd.to_numeric(sample_df_clean['loan_term'], errors='coerce')


In [10]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)

72.76152419009561


In [11]:
# Convert loacombined_loan_to_value_ration_term to float and remove NaN
sample_df_clean['combined_loan_to_value_ratio'] = pd.to_numeric(sample_df_clean['combined_loan_to_value_ratio'], errors='coerce')
sample_df_clean = sample_df_clean.dropna(subset=['combined_loan_to_value_ratio'])

In [12]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)

74.4053851907255


In [13]:
# Convert interest_rate to float and remove NaN
sample_df_clean['interest_rate'] = pd.to_numeric(sample_df_clean['interest_rate'], errors='coerce')
sample_df_clean = sample_df_clean.dropna(subset=['interest_rate'])

In [14]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)

93.90781335703883


In [15]:
# Convert loan_term to int and remove NaN
sample_df_clean['property_value'] = pd.to_numeric(sample_df_clean['property_value'], errors='coerce')
sample_df_clean = sample_df_clean.dropna(subset=['property_value'])
sample_df_clean['property_value'] = sample_df_clean['property_value'].astype(int)

In [16]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)

93.9076981274825


In [17]:
# Encoding total_units ordinally and removing Nan
sample_df_clean['total_units'] = sample_df_clean['total_units'].astype(str).str.strip()

category_order = [['1', '2', '3', '4', '5-24', '25-49', '50-99', '>149']]


encoder = OrdinalEncoder(categories=category_order)
sample_df_clean['total_units'] = encoder.fit_transform(
    sample_df_clean[['total_units']]
).astype(int)

In [18]:
# Encoding applicant_age and co_applicant age and removing Nan
sample_df_clean['applicant_age'] = sample_df_clean['applicant_age'].replace('8888', np.nan)
sample_df_clean = sample_df_clean.dropna(subset=['applicant_age'])
age_order = [['<25', '25-34', '35-44', '45-54', '55-64', '65-74', '>74']]
encoder = OrdinalEncoder(categories=age_order)
sample_df_clean['applicant_age'] = encoder.fit_transform(sample_df_clean[['applicant_age']]).astype(int)


sample_df_clean['co_applicant_age'] = sample_df_clean['co_applicant_age'].replace(['8888', '9999'], np.nan)
sample_df_clean = sample_df_clean.dropna(subset=['co_applicant_age'])
age_order = [['<25', '25-34', '35-44', '45-54', '55-64', '65-74', '>74']]
encoder = OrdinalEncoder(categories=age_order)
sample_df_clean['co_applicant_age'] = encoder.fit_transform(sample_df_clean[['co_applicant_age']]).astype(int)


In [19]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)

95.14183652027737


In [20]:
#Encode debt_to_income_ratio
sample_df_clean['debt_to_income_ratio'] = sample_df_clean['debt_to_income_ratio'].apply(parse_dti)

# Convert to integer if no NaNs remain, otherwise keep float
if sample_df_clean['debt_to_income_ratio'].isna().sum() == 0:
    sample_df_clean['debt_to_income_ratio'] = sample_df_clean['debt_to_income_ratio'].astype(int)

print(sample_df_clean['debt_to_income_ratio'].unique())



[55 25 44 41 40 45 33 20 36 47 37 46 38 39 42 49 43 48 60]


In [21]:
sample_df_clean['derived_race'] = sample_df_clean['derived_race'].astype(str).str.strip()
race_dummies = pd.get_dummies(sample_df_clean['derived_race'], prefix='derived_race', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_race']), race_dummies], axis=1)


In [22]:
sample_df_clean['derived_loan_product_type'] = sample_df_clean['derived_loan_product_type'].astype(str).str.strip()
loan_type_dummies = pd.get_dummies(sample_df_clean['derived_loan_product_type'], prefix='loan_product_type', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_loan_product_type']), loan_type_dummies], axis=1)


In [23]:
sample_df_clean['derived_ethnicity'] = sample_df_clean['derived_ethnicity'].astype(str).str.strip()
ethnicity_dummies = pd.get_dummies(sample_df_clean['derived_ethnicity'], prefix='derived_ethnicity', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_ethnicity']), ethnicity_dummies], axis=1)


In [24]:
sample_df_clean['derived_sex'] = sample_df_clean['derived_sex'].astype(str).str.strip()
sex_dummies = pd.get_dummies(sample_df_clean['derived_sex'], prefix='derived_sex', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_sex']), sex_dummies], axis=1)


In [25]:
sample_df_clean['derived_dwelling_category'] = sample_df_clean['derived_dwelling_category'].astype(str).str.strip()
dwelling_dummies = pd.get_dummies(sample_df_clean['derived_dwelling_category'], prefix='dwelling_category', drop_first=True)
sample_df_clean = pd.concat([sample_df_clean.drop(columns=['derived_dwelling_category']), dwelling_dummies], axis=1)


In [26]:
sample_df_clean = sample_df_clean.dropna(subset=['conforming_loan_limit'])
sample_df_clean['conforming_loan_limit'] = sample_df_clean['conforming_loan_limit'].map({'C': 1,'NC': 0})
sample_df_clean['conforming_loan_limit'] = sample_df_clean['conforming_loan_limit'].astype(bool)
sample_df_clean = sample_df_clean.dropna(subset=['conforming_loan_limit'])



In [27]:
df_1 = sample_df_clean[sample_df_clean['action_taken'] == 1]
df_0 = sample_df_clean[sample_df_clean['action_taken'] == 0]

# Downsample majority class to match minority class size
df_1_balanced = resample(df_1, replace=False, n_samples=len(df_0), random_state=42)

# Combine and shuffle
sample_df_clean = pd.concat([df_1_balanced, df_0]).sample(frac=1, random_state=42).reset_index(drop=True)


df_1 = sample_df_clean[sample_df_clean['applicant_sex'] == 1]
df_0 = sample_df_clean[sample_df_clean['applicant_sex'] == 0]

# Downsample majority class to match minority class size
df_0_balanced = resample(df_0, replace=False, n_samples=len(df_1), random_state=42)

# Combine and shuffle
sample_df_clean = pd.concat([df_0_balanced, df_1]).sample(frac=1, random_state=42).reset_index(drop=True)

In [28]:
Y = sample_df_clean['action_taken'].astype(int)
Y_sen = sample_df_clean['applicant_sex'].astype(int)
X =sample_df_clean.drop(columns=["action_taken", "applicant_sex"])


In [29]:
nan_counts = X.isna().sum()
print(nan_counts)

derived_msa_md                                              0
conforming_loan_limit                                       0
loan_type                                                   0
loan_purpose                                                0
lien_status                                                 0
                                                           ..
derived_ethnicity_Not Hispanic or Latino                    0
derived_sex_Joint                                           0
derived_sex_Male                                            0
dwelling_category_Single Family (1-4 Units):Manufactured    0
dwelling_category_Single Family (1-4 Units):Site-Built      0
Length: 71, dtype: int64


In [30]:
print((sample_df_clean['action_taken'] == 1).mean() * 100)

49.87515605493134


In [34]:
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

mi = mutual_info_classif(X, Y_sen, discrete_features='auto')
mi_scores = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_scores.head(70))  # top 20 most informative

co_applicant_sex                        0.405866
applicant_race_1                        0.026428
derived_race_Race Not Available         0.019214
property_value                          0.016828
derived_ethnicity_Hispanic or Latino    0.016699
                                          ...   
occupancy_type                          0.000000
aus_1                                   0.000000
submission_of_application               0.000000
co_applicant_age                        0.000000
applicant_age                           0.000000
Length: 70, dtype: float64


In [ ]:
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

mi = mutual_info_classif(X, Y, discrete_features='auto')
mi_scores = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_scores.head(70))  # top 20 most informativea

hoepa_status                         0.532489
tract_minority_population_percent    0.125572
tract_owner_occupied_units           0.118425
tract_one_to_four_family_homes       0.107090
ffiec_msa_md_median_family_income    0.105230
                                       ...   
aus_1                                0.000000
occupancy_type                       0.000000
construction_method                  0.000000
other_nonamortizing_features         0.000000
balloon_payment                      0.000000
Length: 70, dtype: float64


In [32]:
from IPython.display import display, HTML

mi_scores_df = mi_scores.reset_index()
mi_scores_df.columns = ['Feature', 'Mutual Information']

display(HTML(
    mi_scores_df.to_html(index=False)
    .replace('<table border="1" class="dataframe">',
             '<table border="1" class="dataframe" style="display:block; max-height:300px; overflow-y:scroll;">')
))


Feature,Mutual Information
hoepa_status,0.532489
tract_minority_population_percent,0.125572
tract_owner_occupied_units,0.118425
tract_one_to_four_family_homes,0.107090
ffiec_msa_md_median_family_income,0.105230
tract_population,0.095303
tract_to_msa_income_percentage,0.086813
tract_median_age_of_housing_units,0.075196
derived_msa_md,0.050805
property_value,0.038359


In [33]:
pct_Y = (Y == 1).mean() * 100
pct_Y_sen_train = (Y_sen == 1).mean() * 100

print(f"Y_train {pct_Y:.2f}% are 1")
print(f"Y_test: {pct_Y_sen_train:.2f}% are 1")


Y_train 49.88% are 1
Y_test: 50.00% are 1
